In [1]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator # pyright: ignore[reportMissingImports]
from tensorflow.keras.models import Sequential # pyright: ignore[reportMissingImports]
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout # pyright: ignore[reportMissingImports]
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint # pyright: ignore[reportMissingImports]

In [2]:
base_dir = os.path.join('Data', 'images')  # contains cats/ and dogs/

In [3]:
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 80% train, 20% validation
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [4]:
train_data = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 559 images belonging to 2 classes.


In [5]:
val_data = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 138 images belonging to 2 classes.


In [6]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

c:\Users\nanus\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,169,089 (42.61 MB)

 Trainable params: 11,169,089 (42.61 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True)

In [9]:
history = model.fit(
    train_data,
    epochs=25,
    validation_data=val_data,
    callbacks=[early_stop, checkpoint]
)

c:\Users\nanus\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.5408 - loss: 1.0683

18/18 ━━━━━━━━━━━━━━━━━━━━ 39s 2s/step - accuracy: 0.5331 - loss: 0.8808 - val_accuracy: 0.5000 - val_loss: 0.6995
Epoch 2/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5286 - loss: 0.6901

18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.5027 - loss: 0.6907 - val_accuracy: 0.5870 - val_loss: 0.6911
Epoch 3/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5170 - loss: 0.6988 - val_accuracy: 0.5870 - val_loss: 0.6922
Epoch 4/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.5689 - loss: 0.6923 - val_accuracy: 0.5435 - val_loss: 0.6860
Epoch 5/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5564 - loss: 0.6851 - val_accuracy: 0.5435 - val_loss: 0.6977
Epoch 6/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5725 - loss: 0.6888 - val_accuracy: 0.5725 - val_loss: 0.6906
Epoch 7/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.5850 - loss: 0.6798 - val_accuracy: 0.5652 - val_loss: 0.6838
Epoch 8/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.6059 - loss: 0.6798

18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.6118 - loss: 0.6808 - val_accuracy: 0.6377 - val_loss: 0.6784
Epoch 9/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.6208 - loss: 0.6788 - val_accuracy: 0.6232 - val_loss: 0.6754
Epoch 10/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.5975 - loss: 0.6775 - val_accuracy: 0.6304 - val_loss: 0.6677
Epoch 11/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.6011 - loss: 0.6735 - val_accuracy: 0.6087 - val_loss: 0.6872
Epoch 12/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 23s 1s/step - accuracy: 0.6047 - loss: 0.6640 - val_accuracy: 0.6087 - val_loss: 0.6844
Epoch 13/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.6243 - loss: 0.6617 - val_accuracy: 0.6232 - val_loss: 0.6979
Epoch 14/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.6494 - loss: 0.6536 - val_accuracy: 0.5435 - val_loss: 0.6873
Epoch 15/25
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.5667 - loss: 0.6775

18/18 ━━━━━━━━━━━━━━━━━━━━ 24s 1s/step - accuracy: 0.5635 - loss: 0.6771 - val_accuracy: 0.6449 - val_loss: 0.6680
